# Text Fabric Setup
We begin by importing the lexemes for our Hebrew corpus.
We work with lexemes so we don't need to worry about tokenizing.

In [1]:
from tf.fabric import Fabric

TF = Fabric(locations='../text-fabric-data/bhsa', modules='tf/2017', silent=True)
api = TF.load('''
	g_lex_utf8
''')

  0.00s loading features ...
   |     0.22s B g_lex_utf8           from /home/jcuenod/Programming/text-fabric-data/bhsa/tf/2017
  4.51s All features loaded/computed - for details use loadLog()


First we'll set up a helper function that will help us count occurrences.

In [2]:
def increment_key(d, i, key):
    if key in i:
        d[ i[key] ]["count"] += 1
    else:
        i[key] = len(d)
        d.append({
            "lexeme": key,
            "count": 1
        })

Now we'll run through every word and count the lexeme occurrences.

In [3]:
lexeme_key_indices = {}
lexeme_dict = []
corpus_size = 0
for n in api.F.otype.s('word'):
    key = api.F.g_lex_utf8.v(n)
    increment_key(lexeme_dict, lexeme_key_indices, key)
    corpus_size += 1

Now let's see what our lexeme_dict contains

In [4]:
lexeme_dict[0:10]

[{'lexeme': 'בְּ', 'count': 6588},
 {'lexeme': 'רֵאשִׁית', 'count': 49},
 {'lexeme': 'בָּרָא', 'count': 9},
 {'lexeme': 'אֱלֹה', 'count': 2389},
 {'lexeme': 'אֵת', 'count': 1171},
 {'lexeme': 'הַ', 'count': 17008},
 {'lexeme': 'שָּׁמַי', 'count': 225},
 {'lexeme': 'וְ', 'count': 24303},
 {'lexeme': 'הָ', 'count': 6633},
 {'lexeme': 'אָרֶץ', 'count': 1350}]

# Let's Analyse "YHWH" (יְהוָה)

## 1. Find Frequencies

In this for loop, we find all the words that are collocated with "יְהוָה" (that is, words that are found in the same clause).

Note: by changing the `WORD_TO_STUDY` at this point, we can do the equivalent analysis on other terms.

In [5]:
# WORD_TO_STUDY = "אָרֶץ"
# WORD_TO_STUDY = "שָּׁמַי"
WORD_TO_STUDY = "יְהוָה"

collocation_key_indices = {}
collocation_dict = []
for n in api.F.otype.s('word'):
    if api.F.g_lex_utf8.v(n) != WORD_TO_STUDY: continue
    relevant_clause = api.L.u(n, otype='clause')[0]
    for nn in api.L.d(relevant_clause, otype='word'):
        if n == nn: continue
        key = api.F.g_lex_utf8.v(nn)
        increment_key(collocation_dict, collocation_key_indices, key)

Now let's see what the most common collocations were. Remember we are not done at this point, in the next step we will weight these frequencies.

In [6]:
sorted_collocations = sorted(collocation_dict, key=lambda x: x["count"], reverse=True)
sorted_collocations[0:10]

[{'lexeme': 'וַ', 'count': 1100},
 {'lexeme': 'אֶת', 'count': 907},
 {'lexeme': 'אֱלֹה', 'count': 871},
 {'lexeme': 'הַ', 'count': 862},
 {'lexeme': 'וְ', 'count': 847},
 {'lexeme': 'בְּ', 'count': 685},
 {'lexeme': 'אֶל', 'count': 638},
 {'lexeme': 'כִּי', 'count': 618},
 {'lexeme': 'אֲשֶׁר', 'count': 580},
 {'lexeme': 'לְ', 'count': 414}]

## 2. Weight Frequencies

We will weight these frequencies using the assocation measure called MI. "MI" comes from "mutual information" (cf. http://www.collocations.de/AM/)

To calculate MI, we need these four values:
- Corpus Size
- Node (CMJM/) occurrences
- Collocate (?) occurrences
- Node + Collocate occurrences (i.e. "observed")

The MI value is then defined by the following function:
```
MI = log2(observed/expected)
```

To calculate "expected" and then MI, we need these functions:

In [7]:
def calculateExpected(node, collocate, corpus_size):
    return (node * collocate) / corpus_size

from math import log2, pow
def calculateMI(observed, expected):
    return log2(observed/expected)

The `calculateMI` function above favours collocations in which words occur infrequently.
This means that hapaxes are automatically given high MI values.

Apparently MI2 is better at dealing with this problem (cf. http://www.collocations.de/AM/). The solution offered by MI2 is to do the calculation as:

```
mi2 = log2(pow(observed,2)/expected)
```
   
In my experience, however, this unduly privileged excessively common lexemes (such as ה and ו). For this reason, it seems better to me to filter out collocations where the other word occurs at very low frequencies. The number is arbitrary but even 2 produces better results. It is helpful to bear in mind that examining collocations is only going to be useful if we have a few occurrences of the collocation so I have settled on 5 at this point.

In [8]:
MIN_COLLOCATIONS = 5
filtered_collocations = list(filter(lambda x: x["count"] > MIN_COLLOCATIONS, collocation_dict))

So now let's add that data to our collocation information

In [9]:
collocation_mi = []
for i in range(len(filtered_collocations)):
    key = filtered_collocations[i]["lexeme"]
    new_entry = {
        "lexeme": key,
        "count": filtered_collocations[i]["count"]
    }
    new_entry["expected"] = calculateExpected(
        node=lexeme_dict[ lexeme_key_indices[WORD_TO_STUDY] ]["count"],
        collocate=lexeme_dict[ lexeme_key_indices[key] ]["count"],
        corpus_size=corpus_size)
    new_entry["MI"] = calculateMI(new_entry["count"], new_entry["expected"])
    collocation_mi.append(new_entry)

## 3. Sort Collocations by MI Weight

Let's sort collocations by MI and see what the highest MIs are:

In [10]:
sorted_collocations_mi = sorted(collocation_mi, key=lambda x: x["MI"], reverse=True)
sorted_collocations_mi[0:10]

[{'lexeme': 'מְשִׁיחַ',
  'count': 11,
  'expected': 0.15983721846107685,
  'MI': 6.104756331308513},
 {'lexeme': 'מַלְאַךְ',
  'count': 58,
  'expected': 0.9323837743562815,
  'MI': 5.958985191574977},
 {'lexeme': 'דְבַר',
  'count': 159,
  'expected': 2.6905931774281266,
  'MI': 5.884958685924933},
 {'lexeme': 'הֵיכַל',
  'count': 29,
  'expected': 0.4928314235883202,
  'MI': 5.8788148428909945},
 {'lexeme': 'שָּׂטָן',
  'count': 13,
  'expected': 0.2264360594865255,
  'MI': 5.8432640902831245},
 {'lexeme': 'בָּרֲך',
  'count': 16,
  'expected': 0.2930349005119742,
  'MI': 5.770855594755075},
 {'lexeme': 'חַגַּי',
  'count': 8,
  'expected': 0.1465174502559871,
  'MI': 5.770855594755075},
 {'lexeme': 'גֹּף',
  'count': 7,
  'expected': 0.13319768205089735,
  'MI': 5.715714040562614},
 {'lexeme': 'צְבָא',
  'count': 252,
  'expected': 4.795116553832305,
  'MI': 5.715714040562614},
 {'lexeme': 'נְאֻם',
  'count': 260,
  'expected': 4.994913076908651,
  'MI': 5.701908241037583}]

## 4. View Results

These results are, according to the MI measure the most meaningful assocations with the term studied above. Remember that we adjusted the collocations, removing infrequent ones so this remains an adjusted MI. Still, the results are intriguing and the last result displayed* illustrates the potential of furthering this study with more than three or more lexemes per collocation.

**The last result displayed (the 20th), כֹּה, is clearly significant because of the phrase "כֹּה אָמַר יהוה" whereas אָמַר is simply too common outside of collocations with יהוה.*

In [11]:
from tf.extra.bhsa import Bhsa
B = Bhsa(api, 'search', version='2017')

for found in sorted_collocations_mi[0:20]:
    print(found)
    best_lexeme = found["lexeme"]
    results = B.search('''
verse
    clause
        word g_lex_utf8={lex1}
        word g_lex_utf8={lex2}
'''.format(lex1=WORD_TO_STUDY, lex2=best_lexeme))

    B.table(results, end=10)

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="{CORPUS.upper()} feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 5.4.0</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>

{'lexeme': 'מְשִׁיחַ', 'count': 11, 'expected': 0.15983721846107685, 'MI': 6.104756331308513}
  1.26s 11 results


n | verse | clause | word | word
--- | --- | --- | --- | ---
1|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=24&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 24:7">1_Samuel 24:7</a> <span class="hb">וַיֹּ֨אמֶר לַאֲנָשָׁ֜יו חָלִ֧ילָה לִּ֣י מֵֽיהוָ֗ה אִם־אֶעֱשֶׂה֩ אֶת־הַדָּבָ֨ר הַזֶּ֤ה לַֽאדֹנִי֙ לִמְשִׁ֣יחַ יְהוָ֔ה לִשְׁלֹ֥חַ יָדִ֖י בֹּ֑ו כִּֽי־מְשִׁ֥יחַ יְהוָ֖ה הֽוּא׃ </span>|<span class="hb">אִם־אֶעֱשֶׂה֩ אֶת־הַדָּבָ֨ר הַזֶּ֤ה לַֽאדֹנִי֙ לִמְשִׁ֣יחַ יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">מְשִׁ֣יחַ </span>
2|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=24&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 24:7">1_Samuel 24:7</a> <span class="hb">וַיֹּ֨אמֶר לַאֲנָשָׁ֜יו חָלִ֧ילָה לִּ֣י מֵֽיהוָ֗ה אִם־אֶעֱשֶׂה֩ אֶת־הַדָּבָ֨ר הַזֶּ֤ה לַֽאדֹנִי֙ לִמְשִׁ֣יחַ יְהוָ֔ה לִשְׁלֹ֥חַ יָדִ֖י בֹּ֑ו כִּֽי־מְשִׁ֥יחַ יְהוָ֖ה הֽוּא׃ </span>|<span class="hb">כִּֽי־מְשִׁ֥יחַ יְהוָ֖ה הֽוּא׃ </span>|<span class="hb">יְהוָ֖ה </span>|<span class="hb">מְשִׁ֥יחַ </span>
3|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=24&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 24:11">1_Samuel 24:11</a> <span class="hb">הִנֵּה֩ הַיֹּ֨ום הַזֶּ֜ה רָא֣וּ עֵינֶ֗יךָ אֵ֣ת אֲשֶׁר־נְתָנְךָ֩ יְהוָ֨ה׀ הַיֹּ֤ום׀ בְּיָדִי֙ בַּמְּעָרָ֔ה וְאָמַ֥ר לַהֲרָגֲךָ֖ וַתָּ֣חָס עָלֶ֑יךָ וָאֹמַ֗ר לֹא־אֶשְׁלַ֤ח יָדִי֙ בַּֽאדֹנִ֔י כִּי־מְשִׁ֥יחַ יְהוָ֖ה הֽוּא׃ </span>|<span class="hb">כִּי־מְשִׁ֥יחַ יְהוָ֖ה הֽוּא׃ </span>|<span class="hb">יְהוָ֖ה </span>|<span class="hb">מְשִׁ֥יחַ </span>
4|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=26&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 26:9">1_Samuel 26:9</a> <span class="hb">וַיֹּ֧אמֶר דָּוִ֛ד אֶל־אֲבִישַׁ֖י אַל־תַּשְׁחִיתֵ֑הוּ כִּ֠י מִ֣י שָׁלַ֥ח יָדֹ֛ו בִּמְשִׁ֥יחַ יְהוָ֖ה וְנִקָּֽה׃ פ </span>|<span class="hb">כִּ֠י מִ֣י שָׁלַ֥ח יָדֹ֛ו בִּמְשִׁ֥יחַ יְהוָ֖ה </span>|<span class="hb">יְהוָ֖ה </span>|<span class="hb">מְשִׁ֥יחַ </span>
5|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=26&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 26:11">1_Samuel 26:11</a> <span class="hb">חָלִ֤ילָה לִּי֙ מֵֽיהוָ֔ה מִשְּׁלֹ֥חַ יָדִ֖י בִּמְשִׁ֣יחַ יְהוָ֑ה וְ֠עַתָּה קַח־נָ֨א אֶֽת־הַחֲנִ֜ית אֲשֶׁ֧ר מְרַאֲשֹׁתָ֛יו וְאֶת־צַפַּ֥חַת הַמַּ֖יִם וְנֵ֥לֲכָה לָּֽנוּ׃ </span>|<span class="hb">מִשְּׁלֹ֥חַ יָדִ֖י בִּמְשִׁ֣יחַ יְהוָ֑ה </span>|<span class="hb">יְהוָ֑ה </span>|<span class="hb">מְשִׁ֣יחַ </span>
6|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=26&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 26:16">1_Samuel 26:16</a> <span class="hb">לֹא־טֹ֞וב הַדָּבָ֣ר הַזֶּה֮ אֲשֶׁ֣ר עָשִׂיתָ֒ חַי־יְהוָ֗ה כִּ֤י בְנֵי־מָ֨וֶת֙ אַתֶּ֔ם אֲשֶׁ֧ר לֹֽא־שְׁמַרְתֶּ֛ם עַל־אֲדֹנֵיכֶ֖ם עַל־מְשִׁ֣יחַ יְהוָ֑ה וְעַתָּ֣ה׀ רְאֵ֗ה אֵֽי־חֲנִ֥ית הַמֶּ֛לֶךְ וְאֶת־צַפַּ֥חַת הַמַּ֖יִם אֲשֶׁ֥ר מְרַאֲשֹׁתָֽיו׃</span>|<span class="hb">אֲשֶׁ֧ר לֹֽא־שְׁמַרְתֶּ֛ם עַל־אֲדֹנֵיכֶ֖ם עַל־מְשִׁ֣יחַ יְהוָ֑ה </span>|<span class="hb">יְהוָ֑ה </span>|<span class="hb">מְשִׁ֣יחַ </span>
7|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=26&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 26:23">1_Samuel 26:23</a> <span class="hb">וַֽיהוָה֙ יָשִׁ֣יב לָאִ֔ישׁ אֶת־צִדְקָתֹ֖ו וְאֶת־אֱמֻנָתֹ֑ו אֲשֶׁר֩ נְתָנְךָ֙ יְהוָ֤ה׀ הַיֹּום֙ בְּיָ֔ד וְלֹ֣א אָבִ֔יתִי לִשְׁלֹ֥חַ יָדִ֖י בִּמְשִׁ֥יחַ יְהוָֽה׃ </span>|<span class="hb">לִשְׁלֹ֥חַ יָדִ֖י בִּמְשִׁ֥יחַ יְהוָֽה׃ </span>|<span class="hb">יְהוָֽה׃ </span>|<span class="hb">מְשִׁ֥יחַ </span>
8|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=1&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 1:14">2_Samuel 1:14</a> <span class="hb">וַיֹּ֥אמֶר אֵלָ֖יו דָּוִ֑ד אֵ֚יךְ לֹ֣א יָרֵ֔אתָ לִשְׁלֹ֨חַ֙ יָֽדְךָ֔ לְשַׁחֵ֖ת אֶת־מְשִׁ֥יחַ יְהוָֽה׃ </span>|<span class="hb">לְשַׁחֵ֖ת אֶת־מְשִׁ֥יחַ יְהוָֽה׃ </span>|<span class="hb">יְהוָֽה׃ </span>|<span class="hb">מְשִׁ֥יחַ </span>
9|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=1&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 1:16">2_Samuel 1:16</a> <span class="hb">וַיֹּ֤אמֶר אֵלָיו֙ דָּוִ֔ד דָּמְךָ֖ עַל־רֹאשֶׁ֑ךָ כִּ֣י פִ֗יךָ עָנָ֤ה בְךָ֙ לֵאמֹ֔ר אָנֹכִ֥י מֹתַ֖תִּי אֶת־מְשִׁ֥יחַ יְהוָֽה׃ ס </span>|<span class="hb">אָנֹכִ֥י מֹתַ֖תִּי אֶת־מְשִׁ֥יחַ יְהוָֽה׃ ס </span>|<span class="hb">יְהוָֽה׃ ס </span>|<span class="hb">מְשִׁ֥יחַ </span>
10|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_II&chapter=19&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Samuel 19:22">2_Samuel 19:22</a> <span class="hb">וַיַּ֨עַן אֲבִישַׁ֤י בֶּן־צְרוּיָה֙ וַיֹּ֔אמֶר הֲתַ֣חַת זֹ֔את לֹ֥א יוּמַ֖ת שִׁמְעִ֑י כִּ֥י קִלֵּ֖ל אֶת־מְשִׁ֥יחַ יְהוָֽה׃ ס </span>|<span class="hb">כִּ֥י קִלֵּ֖ל אֶת־מְשִׁ֥יחַ יְהוָֽה׃ ס </span>|<span class="hb">יְהוָֽה׃ ס </span>|<span class="hb">מְשִׁ֥יחַ </span>

{'lexeme': 'מַלְאַךְ', 'count': 58, 'expected': 0.9323837743562815, 'MI': 5.958985191574977}
  1.32s 58 results


n | verse | clause | word | word
--- | --- | --- | --- | ---
1|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=16&verse=7&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 16:7">Genesis 16:7</a> <span class="hb">וַֽיִּמְצָאָ֞הּ מַלְאַ֧ךְ יְהוָ֛ה עַל־עֵ֥ין הַמַּ֖יִם בַּמִּדְבָּ֑ר עַל־הָעַ֖יִן בְּדֶ֥רֶךְ שֽׁוּר׃ </span>|<span class="hb">וַֽיִּמְצָאָ֞הּ מַלְאַ֧ךְ יְהוָ֛ה עַל־עֵ֥ין הַמַּ֖יִם בַּמִּדְבָּ֑ר עַל־הָעַ֖יִן בְּדֶ֥רֶךְ שֽׁוּר׃ </span>|<span class="hb">יְהוָ֛ה </span>|<span class="hb">מַלְאַ֧ךְ </span>
2|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=16&verse=9&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 16:9">Genesis 16:9</a> <span class="hb">וַיֹּ֤אמֶר לָהּ֙ מַלְאַ֣ךְ יְהוָ֔ה שׁ֖וּבִי אֶל־גְּבִרְתֵּ֑ךְ וְהִתְעַנִּ֖י תַּ֥חַת יָדֶֽיהָ׃ </span>|<span class="hb">וַיֹּ֤אמֶר לָהּ֙ מַלְאַ֣ךְ יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">מַלְאַ֣ךְ </span>
3|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=16&verse=10&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 16:10">Genesis 16:10</a> <span class="hb">וַיֹּ֤אמֶר לָהּ֙ מַלְאַ֣ךְ יְהוָ֔ה הַרְבָּ֥ה אַרְבֶּ֖ה אֶת־זַרְעֵ֑ךְ וְלֹ֥א יִסָּפֵ֖ר מֵרֹֽב׃ </span>|<span class="hb">וַיֹּ֤אמֶר לָהּ֙ מַלְאַ֣ךְ יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">מַלְאַ֣ךְ </span>
4|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=16&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 16:11">Genesis 16:11</a> <span class="hb">וַיֹּ֤אמֶר לָהּ֙ מַלְאַ֣ךְ יְהוָ֔ה הִנָּ֥ךְ הָרָ֖ה וְיֹלַ֣דְתְּ בֵּ֑ן וְקָרָ֤את שְׁמֹו֙ יִשְׁמָעֵ֔אל כִּֽי־שָׁמַ֥ע יְהוָ֖ה אֶל־עָנְיֵֽךְ׃ </span>|<span class="hb">וַיֹּ֤אמֶר לָהּ֙ מַלְאַ֣ךְ יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">מַלְאַ֣ךְ </span>
5|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:11">Genesis 22:11</a> <span class="hb">וַיִּקְרָ֨א אֵלָ֜יו מַלְאַ֤ךְ יְהוָה֙ מִן־הַשָּׁמַ֔יִם וַיֹּ֖אמֶר אַבְרָהָ֣ם׀ אַבְרָהָ֑ם וַיֹּ֖אמֶר הִנֵּֽנִי׃ </span>|<span class="hb">וַיִּקְרָ֨א אֵלָ֜יו מַלְאַ֤ךְ יְהוָה֙ מִן־הַשָּׁמַ֔יִם </span>|<span class="hb">יְהוָה֙ </span>|<span class="hb">מַלְאַ֤ךְ </span>
6|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=22&verse=15&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 22:15">Genesis 22:15</a> <span class="hb">וַיִּקְרָ֛א מַלְאַ֥ךְ יְהוָ֖ה אֶל־אַבְרָהָ֑ם שֵׁנִ֖ית מִן־הַשָּׁמָֽיִם׃ </span>|<span class="hb">וַיִּקְרָ֛א מַלְאַ֥ךְ יְהוָ֖ה אֶל־אַבְרָהָ֑ם שֵׁנִ֖ית מִן־הַשָּׁמָֽיִם׃ </span>|<span class="hb">יְהוָ֖ה </span>|<span class="hb">מַלְאַ֥ךְ </span>
7|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=22&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 22:22">Numbers 22:22</a> <span class="hb">וַיִּֽחַר־אַ֣ף אֱלֹהִים֮ כִּֽי־הֹולֵ֣ךְ הוּא֒ וַיִּתְיַצֵּ֞ב מַלְאַ֧ךְ יְהוָ֛ה בַּדֶּ֖רֶךְ לְשָׂטָ֣ן לֹ֑ו וְהוּא֙ רֹכֵ֣ב עַל־אֲתֹנֹ֔ו וּשְׁנֵ֥י נְעָרָ֖יו עִמֹּֽו׃ </span>|<span class="hb">וַיִּתְיַצֵּ֞ב מַלְאַ֧ךְ יְהוָ֛ה בַּדֶּ֖רֶךְ לְשָׂטָ֣ן לֹ֑ו </span>|<span class="hb">יְהוָ֛ה </span>|<span class="hb">מַלְאַ֧ךְ </span>
8|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=22&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 22:23">Numbers 22:23</a> <span class="hb">וַתֵּ֣רֶא הָאָתֹון֩ אֶת־מַלְאַ֨ךְ יְהוָ֜ה נִצָּ֣ב בַּדֶּ֗רֶךְ וְחַרְבֹּ֤ו שְׁלוּפָה֙ בְּיָדֹ֔ו וַתֵּ֤ט הָֽאָתֹון֙ מִן־הַדֶּ֔רֶךְ וַתֵּ֖לֶךְ בַּשָּׂדֶ֑ה וַיַּ֤ךְ בִּלְעָם֙ אֶת־הָ֣אָתֹ֔ון לְהַטֹּתָ֖הּ הַדָּֽרֶךְ׃ </span>|<span class="hb">וַתֵּ֣רֶא הָאָתֹון֩ אֶת־מַלְאַ֨ךְ יְהוָ֜ה </span>|<span class="hb">יְהוָ֜ה </span>|<span class="hb">מַלְאַ֨ךְ </span>
9|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=22&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 22:24">Numbers 22:24</a> <span class="hb">וַֽיַּעֲמֹד֙ מַלְאַ֣ךְ יְהוָ֔ה בְּמִשְׁעֹ֖ול הַכְּרָמִ֑ים גָּדֵ֥ר מִזֶּ֖ה וְגָדֵ֥ר מִזֶּֽה׃ </span>|<span class="hb">וַֽיַּעֲמֹד֙ מַלְאַ֣ךְ יְהוָ֔ה בְּמִשְׁעֹ֖ול הַכְּרָמִ֑ים </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">מַלְאַ֣ךְ </span>
10|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Numeri&chapter=22&verse=25&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Numbers 22:25">Numbers 22:25</a> <span class="hb">וַתֵּ֨רֶא הָאָתֹ֜ון אֶת־מַלְאַ֣ךְ יְהוָ֗ה וַתִּלָּחֵץ֙ אֶל־הַקִּ֔יר וַתִּלְחַ֛ץ אֶת־רֶ֥גֶל בִּלְעָ֖ם אֶל־הַקִּ֑יר וַיֹּ֖סֶף לְהַכֹּתָֽהּ׃ </span>|<span class="hb">וַתֵּ֨רֶא הָאָתֹ֜ון אֶת־מַלְאַ֣ךְ יְהוָ֗ה </span>|<span class="hb">יְהוָ֗ה </span>|<span class="hb">מַלְאַ֣ךְ </span>

...

n | verse | clause | word | word
--- | --- | --- | --- | ---
1|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=6&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 6:24">Deuteronomy 6:24</a> <span class="hb">וַיְצַוֵּ֣נוּ יְהוָ֗ה לַעֲשֹׂות֙ אֶת־כָּל־הַחֻקִּ֣ים הָאֵ֔לֶּה לְיִרְאָ֖ה אֶת־יְהוָ֣ה אֱלֹהֵ֑ינוּ לְטֹ֥וב לָ֨נוּ֙ כָּל־הַיָּמִ֔ים לְחַיֹּתֵ֖נוּ כְּהַיֹּ֥ום הַזֶּֽה׃ </span>|<span class="hb">לְיִרְאָ֖ה אֶת־יְהוָ֣ה אֱלֹהֵ֑ינוּ </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">יִרְאָ֖ה </span>
2|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=10&verse=12&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 10:12">Deuteronomy 10:12</a> <span class="hb">וְעַתָּה֙ יִשְׂרָאֵ֔ל מָ֚ה יְהוָ֣ה אֱלֹהֶ֔יךָ שֹׁאֵ֖ל מֵעִמָּ֑ךְ כִּ֣י אִם־לְ֠יִרְאָה אֶת־יְהוָ֨ה אֱלֹהֶ֜יךָ לָלֶ֤כֶת בְּכָל־דְּרָכָיו֙ וּלְאַהֲבָ֣ה אֹתֹ֔ו וְלַֽעֲבֹד֙ אֶת־יְהוָ֣ה אֱלֹהֶ֔יךָ בְּכָל־לְבָבְךָ֖ וּבְכָל־נַפְשֶֽׁךָ׃ </span>|<span class="hb">לְ֠יִרְאָה אֶת־יְהוָ֨ה אֱלֹהֶ֜יךָ </span>|<span class="hb">יְהוָ֨ה </span>|<span class="hb">יִרְאָה </span>
3|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=14&verse=23&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 14:23">Deuteronomy 14:23</a> <span class="hb">וְאָכַלְתָּ֞ לִפְנֵ֣י׀ יְהוָ֣ה אֱלֹהֶ֗יךָ בַּמָּקֹ֣ום אֲשֶׁר־יִבְחַר֮ לְשַׁכֵּ֣ן שְׁמֹ֣ו שָׁם֒ מַעְשַׂ֤ר דְּגָֽנְךָ֙ תִּֽירֹשְׁךָ֣ וְיִצְהָרֶ֔ךָ וּבְכֹרֹ֥ת בְּקָרְךָ֖ וְצֹאנֶ֑ךָ לְמַ֣עַן תִּלְמַ֗ד לְיִרְאָ֛ה אֶת־יְהוָ֥ה אֱלֹהֶ֖יךָ כָּל־הַיָּמִֽים׃ </span>|<span class="hb">לְיִרְאָ֛ה אֶת־יְהוָ֥ה אֱלֹהֶ֖יךָ כָּל־הַיָּמִֽים׃ </span>|<span class="hb">יְהוָ֥ה </span>|<span class="hb">יִרְאָ֛ה </span>
4|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=17&verse=19&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 17:19">Deuteronomy 17:19</a> <span class="hb">וְהָיְתָ֣ה עִמֹּ֔ו וְקָ֥רָא בֹ֖ו כָּל־יְמֵ֣י חַיָּ֑יו לְמַ֣עַן יִלְמַ֗ד לְיִרְאָה֙ אֶת־יְהוָ֣ה אֱלֹהָ֔יו לִ֠שְׁמֹר אֶֽת־כָּל־דִּבְרֵ֞י הַתֹּורָ֥ה הַזֹּ֛את וְאֶת־הַחֻקִּ֥ים הָאֵ֖לֶּה לַעֲשֹׂתָֽם׃ </span>|<span class="hb">לְיִרְאָה֙ אֶת־יְהוָ֣ה אֱלֹהָ֔יו </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">יִרְאָה֙ </span>
5|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=28&verse=58&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 28:58">Deuteronomy 28:58</a> <span class="hb">אִם־לֹ֨א תִשְׁמֹ֜ר לַעֲשֹׂ֗ות אֶת־כָּל־דִּבְרֵי֙ הַתֹּורָ֣ה הַזֹּ֔את הַכְּתוּבִ֖ים בַּסֵּ֣פֶר הַזֶּ֑ה לְ֠יִרְאָה אֶת־הַשֵּׁ֞ם הַנִּכְבָּ֤ד וְהַנֹּורָא֙ הַזֶּ֔ה אֵ֖ת יְהוָ֥ה אֱלֹהֶֽיךָ׃ </span>|<span class="hb">לְ֠יִרְאָה אֶת־הַשֵּׁ֞ם הַנִּכְבָּ֤ד וְהַנֹּורָא֙ הַזֶּ֔ה אֵ֖ת יְהוָ֥ה אֱלֹהֶֽיךָ׃ </span>|<span class="hb">יְהוָ֥ה </span>|<span class="hb">יִרְאָה </span>
6|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Deuteronomium&chapter=31&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Deuteronomy 31:13">Deuteronomy 31:13</a> <span class="hb">וּבְנֵיהֶ֞ם אֲשֶׁ֣ר לֹֽא־יָדְע֗וּ יִשְׁמְעוּ֙ וְלָ֣מְד֔וּ לְיִרְאָ֖ה אֶת־יְהוָ֣ה אֱלֹהֵיכֶ֑ם כָּל־הַיָּמִ֗ים אֲשֶׁ֨ר אַתֶּ֤ם חַיִּים֙ עַל־הָ֣אֲדָמָ֔ה אֲשֶׁ֨ר אַתֶּ֜ם עֹבְרִ֧ים אֶת־הַיַּרְדֵּ֛ן שָׁ֖מָּה לְרִשְׁתָּֽהּ׃ פ </span>|<span class="hb">לְיִרְאָ֖ה אֶת־יְהוָ֣ה אֱלֹהֵיכֶ֑ם כָּל־הַיָּמִ֗ים </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">יִרְאָ֖ה </span>
7|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=11&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 11:2">Isaiah 11:2</a> <span class="hb">וְנָחָ֥ה עָלָ֖יו ר֣וּחַ יְהוָ֑ה ר֧וּחַ חָכְמָ֣ה וּבִינָ֗ה ר֤וּחַ עֵצָה֙ וּגְבוּרָ֔ה ר֥וּחַ דַּ֖עַת וְיִרְאַ֥ת יְהוָֽה׃ </span>|<span class="hb">וְנָחָ֥ה עָלָ֖יו ר֣וּחַ יְהוָ֑ה ר֧וּחַ חָכְמָ֣ה וּבִינָ֗ה ר֤וּחַ עֵצָה֙ וּגְבוּרָ֔ה ר֥וּחַ דַּ֖עַת וְיִרְאַ֥ת יְהוָֽה׃ </span>|<span class="hb">יְהוָ֑ה </span>|<span class="hb">יִרְאַ֥ת </span>
8|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=11&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 11:2">Isaiah 11:2</a> <span class="hb">וְנָחָ֥ה עָלָ֖יו ר֣וּחַ יְהוָ֑ה ר֧וּחַ חָכְמָ֣ה וּבִינָ֗ה ר֤וּחַ עֵצָה֙ וּגְבוּרָ֔ה ר֥וּחַ דַּ֖עַת וְיִרְאַ֥ת יְהוָֽה׃ </span>|<span class="hb">וְנָחָ֥ה עָלָ֖יו ר֣וּחַ יְהוָ֑ה ר֧וּחַ חָכְמָ֣ה וּבִינָ֗ה ר֤וּחַ עֵצָה֙ וּגְבוּרָ֔ה ר֥וּחַ דַּ֖עַת וְיִרְאַ֥ת יְהוָֽה׃ </span>|<span class="hb">יְהוָֽה׃ </span>|<span class="hb">יִרְאַ֥ת </span>
9|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=11&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 11:3">Isaiah 11:3</a> <span class="hb">וַהֲרִיחֹ֖ו בְּיִרְאַ֣ת יְהוָ֑ה וְלֹֽא־לְמַרְאֵ֤ה עֵינָיו֙ יִשְׁפֹּ֔וט וְלֹֽא־לְמִשְׁמַ֥ע אָזְנָ֖יו יֹוכִֽיחַ׃ </span>|<span class="hb">וַהֲרִיחֹ֖ו בְּיִרְאַ֣ת יְהוָ֑ה </span>|<span class="hb">יְהוָ֑ה </span>|<span class="hb">יִרְאַ֣ת </span>
10|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Jesaia&chapter=33&verse=6&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Isaiah 33:6">Isaiah 33:6</a> <span class="hb">וְהָיָה֙ אֱמוּנַ֣ת עִתֶּ֔יךָ חֹ֥סֶן יְשׁוּעֹ֖ת חָכְמַ֣ת וָדָ֑עַת יִרְאַ֥ת יְהוָ֖ה הִ֥יא אֹוצָרֹֽו׃ ס </span>|<span class="hb">יִרְאַ֥ת יְהוָ֖ה </span>|<span class="hb">יְהוָ֖ה </span>|<span class="hb">יִרְאַ֥ת </span>

{'lexeme': 'עִבְד', 'count': 9, 'expected': 0.2264360594865255, 'MI': 5.312749373584345}
  1.33s 9 results


n | verse | clause | word | word
--- | --- | --- | --- | ---
1|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=10&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 10:8">Exodus 10:8</a> <span class="hb">וַיּוּשַׁ֞ב אֶת־מֹשֶׁ֤ה וְאֶֽת־אַהֲרֹן֙ אֶל־פַּרְעֹ֔ה וַיֹּ֣אמֶר אֲלֵהֶ֔ם לְכ֥וּ עִבְד֖וּ אֶת־יְהוָ֣ה אֱלֹהֵיכֶ֑ם מִ֥י וָמִ֖י הַהֹלְכִֽים׃ </span>|<span class="hb">עִבְד֖וּ אֶת־יְהוָ֣ה אֱלֹהֵיכֶ֑ם </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">עִבְד֖וּ </span>
2|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=10&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 10:11">Exodus 10:11</a> <span class="hb">לֹ֣א כֵ֗ן לְכֽוּ־נָ֤א הַגְּבָרִים֙ וְעִבְד֣וּ אֶת־יְהוָ֔ה כִּ֥י אֹתָ֖הּ אַתֶּ֣ם מְבַקְשִׁ֑ים וַיְגָ֣רֶשׁ אֹתָ֔ם מֵאֵ֖ת פְּנֵ֥י פַרְעֹֽה׃ פ </span>|<span class="hb">וְעִבְד֣וּ אֶת־יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">עִבְד֣וּ </span>
3|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=10&verse=24&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 10:24">Exodus 10:24</a> <span class="hb">וַיִּקְרָ֨א פַרְעֹ֜ה אֶל־מֹשֶׁ֗ה וַיֹּ֨אמֶר֙ לְכוּ֙ עִבְד֣וּ אֶת־יְהוָ֔ה רַ֛ק צֹאנְכֶ֥ם וּבְקַרְכֶ֖ם יֻצָּ֑ג גַּֽם־טַפְּכֶ֖ם יֵלֵ֥ךְ עִמָּכֶֽם׃ </span>|<span class="hb">עִבְד֣וּ אֶת־יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">עִבְד֣וּ </span>
4|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=12&verse=31&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 12:31">Exodus 12:31</a> <span class="hb">וַיִּקְרָא֩ לְמֹשֶׁ֨ה וּֽלְאַהֲרֹ֜ן לַ֗יְלָה וַיֹּ֨אמֶר֙ ק֤וּמוּ צְּאוּ֙ מִתֹּ֣וךְ עַמִּ֔י גַּם־אַתֶּ֖ם גַּם־בְּנֵ֣י יִשְׂרָאֵ֑ל וּלְכ֛וּ עִבְד֥וּ אֶת־יְהוָ֖ה כְּדַבֶּרְכֶֽם׃ </span>|<span class="hb">עִבְד֥וּ אֶת־יְהוָ֖ה </span>|<span class="hb">יְהוָ֖ה </span>|<span class="hb">עִבְד֥וּ </span>
5|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Josua&chapter=24&verse=14&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Joshua 24:14">Joshua 24:14</a> <span class="hb">וְעַתָּ֞ה יְר֧אוּ אֶת־יְהוָ֛ה וְעִבְד֥וּ אֹתֹ֖ו בְּתָמִ֣ים וּבֶֽאֱמֶ֑ת וְהָסִ֣ירוּ אֶת־אֱלֹהִ֗ים אֲשֶׁר֩ עָבְד֨וּ אֲבֹותֵיכֶ֜ם בְּעֵ֤בֶר הַנָּהָר֙ וּבְמִצְרַ֔יִם וְעִבְד֖וּ אֶת־יְהוָֽה׃ </span>|<span class="hb">וְעִבְד֖וּ אֶת־יְהוָֽה׃ </span>|<span class="hb">יְהוָֽה׃ </span>|<span class="hb">עִבְד֖וּ </span>
6|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=2&verse=11&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 2:11">Psalms 2:11</a> <span class="hb">עִבְד֣וּ אֶת־יְהוָ֣ה בְּיִרְאָ֑ה וְ֝גִ֗ילוּ בִּרְעָדָֽה׃ </span>|<span class="hb">עִבְד֣וּ אֶת־יְהוָ֣ה בְּיִרְאָ֑ה </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">עִבְד֣וּ </span>
7|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=100&verse=2&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 100:2">Psalms 100:2</a> <span class="hb">עִבְד֣וּ אֶת־יְהוָ֣ה בְּשִׂמְחָ֑ה בֹּ֥אוּ לְ֝פָנָ֗יו בִּרְנָנָֽה׃ </span>|<span class="hb">עִבְד֣וּ אֶת־יְהוָ֣ה בְּשִׂמְחָ֑ה </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">עִבְד֣וּ </span>
8|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Chronica_II&chapter=30&verse=8&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Chronicles 30:8">2_Chronicles 30:8</a> <span class="hb">עַתָּ֕ה אַל־תַּקְשׁ֥וּ עָרְפְּכֶ֖ם כַּאֲבֹותֵיכֶ֑ם תְּנוּ־יָ֣ד לַיהוָ֗ה וּבֹ֤אוּ לְמִקְדָּשֹׁו֙ אֲשֶׁ֣ר הִקְדִּ֣ישׁ לְעֹולָ֔ם וְעִבְדוּ֙ אֶת־יְהוָ֣ה אֱלֹהֵיכֶ֔ם וְיָשֹׁ֥ב מִכֶּ֖ם חֲרֹ֥ון אַפֹּֽו׃ </span>|<span class="hb">וְעִבְדוּ֙ אֶת־יְהוָ֣ה אֱלֹהֵיכֶ֔ם </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">עִבְדוּ֙ </span>
9|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Chronica_II&chapter=35&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Chronicles 35:3">2_Chronicles 35:3</a> <span class="hb">וַיֹּ֣אמֶר לַ֠לְוִיִּם הַמְּבִינִ֨ים לְכָל־יִשְׂרָאֵ֜ל הַקְּדֹושִׁ֣ים לַיהוָ֗ה תְּנ֤וּ אֶת־אֲרֹון־הַקֹּ֨דֶשׁ֙ בַּ֠בַּיִת אֲשֶׁ֨ר בָּנָ֜ה שְׁלֹמֹ֤ה בֶן־דָּוִיד֙ מֶ֣לֶךְ יִשְׂרָאֵ֔ל אֵין־לָכֶ֥ם מַשָּׂ֖א בַּכָּתֵ֑ף עַתָּ֗ה עִבְדוּ֙ אֶת־יְהוָ֣ה אֱלֹֽהֵיכֶ֔ם וְאֵ֖ת עַמֹּ֥ו יִשְׂרָאֵֽל׃ </span>|<span class="hb">עַתָּ֗ה עִבְדוּ֙ אֶת־יְהוָ֣ה אֱלֹֽהֵיכֶ֔ם וְאֵ֖ת עַמֹּ֥ו יִשְׂרָאֵֽל׃ </span>|<span class="hb">יְהוָ֣ה </span>|<span class="hb">עִבְדוּ֙ </span>

{'lexeme': 'כֹּה', 'count': 255, 'expected': 6.473407347673612, 'MI': 5.299828146645449}
  1.18s 255 results


n | verse | clause | word | word
--- | --- | --- | --- | ---
1|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=4&verse=22&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 4:22">Exodus 4:22</a> <span class="hb">וְאָמַרְתָּ֖ אֶל־פַּרְעֹ֑ה כֹּ֚ה אָמַ֣ר יְהוָ֔ה בְּנִ֥י בְכֹרִ֖י יִשְׂרָאֵֽל׃ </span>|<span class="hb">כֹּ֚ה אָמַ֣ר יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">כֹּ֚ה </span>
2|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=5&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 5:1">Exodus 5:1</a> <span class="hb">וְאַחַ֗ר בָּ֚אוּ מֹשֶׁ֣ה וְאַהֲרֹ֔ן וַיֹּאמְר֖וּ אֶל־פַּרְעֹ֑ה כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י יִשְׂרָאֵ֔ל שַׁלַּח֙ אֶת־עַמִּ֔י וְיָחֹ֥גּוּ לִ֖י בַּמִּדְבָּֽר׃ </span>|<span class="hb">כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י יִשְׂרָאֵ֔ל </span>|<span class="hb">יְהוָה֙ </span>|<span class="hb">כֹּֽה־</span>
3|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=7&verse=17&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 7:17">Exodus 7:17</a> <span class="hb">כֹּ֚ה אָמַ֣ר יְהוָ֔ה בְּזֹ֣את תֵּדַ֔ע כִּ֖י אֲנִ֣י יְהוָ֑ה הִנֵּ֨ה אָנֹכִ֜י מַכֶּ֣ה׀ בַּמַּטֶּ֣ה אֲשֶׁר־בְּיָדִ֗י עַל־הַמַּ֛יִם אֲשֶׁ֥ר בַּיְאֹ֖ר וְנֶהֶפְכ֥וּ לְדָֽם׃ </span>|<span class="hb">כֹּ֚ה אָמַ֣ר יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">כֹּ֚ה </span>
4|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=7&verse=26&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 7:26">Exodus 7:26</a> <span class="hb">וַיֹּ֤אמֶר יְהוָה֙ אֶל־מֹשֶׁ֔ה בֹּ֖א אֶל־פַּרְעֹ֑ה וְאָמַרְתָּ֣ אֵלָ֗יו כֹּ֚ה אָמַ֣ר יְהוָ֔ה שַׁלַּ֥ח אֶת־עַמִּ֖י וְיַֽעַבְדֻֽנִי׃ </span>|<span class="hb">כֹּ֚ה אָמַ֣ר יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">כֹּ֚ה </span>
5|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=8&verse=16&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 8:16">Exodus 8:16</a> <span class="hb">וַיֹּ֨אמֶר יְהוָ֜ה אֶל־מֹשֶׁ֗ה הַשְׁכֵּ֤ם בַּבֹּ֨קֶר֙ וְהִתְיַצֵּב֙ לִפְנֵ֣י פַרְעֹ֔ה הִנֵּ֖ה יֹוצֵ֣א הַמָּ֑יְמָה וְאָמַרְתָּ֣ אֵלָ֗יו כֹּ֚ה אָמַ֣ר יְהוָ֔ה שַׁלַּ֥ח עַמִּ֖י וְיַֽעַבְדֻֽנִי׃ </span>|<span class="hb">כֹּ֚ה אָמַ֣ר יְהוָ֔ה </span>|<span class="hb">יְהוָ֔ה </span>|<span class="hb">כֹּ֚ה </span>
6|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=9&verse=1&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 9:1">Exodus 9:1</a> <span class="hb">וַיֹּ֤אמֶר יְהוָה֙ אֶל־מֹשֶׁ֔ה בֹּ֖א אֶל־פַּרְעֹ֑ה וְדִבַּרְתָּ֣ אֵלָ֗יו כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י הָֽעִבְרִ֔ים שַׁלַּ֥ח אֶת־עַמִּ֖י וְיַֽעַבְדֻֽנִי׃ </span>|<span class="hb">כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י הָֽעִבְרִ֔ים </span>|<span class="hb">יְהוָה֙ </span>|<span class="hb">כֹּֽה־</span>
7|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=9&verse=13&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 9:13">Exodus 9:13</a> <span class="hb">וַיֹּ֤אמֶר יְהוָה֙ אֶל־מֹשֶׁ֔ה הַשְׁכֵּ֣ם בַּבֹּ֔קֶר וְהִתְיַצֵּ֖ב לִפְנֵ֣י פַרְעֹ֑ה וְאָמַרְתָּ֣ אֵלָ֗יו כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י הָֽעִבְרִ֔ים שַׁלַּ֥ח אֶת־עַמִּ֖י וְיַֽעַבְדֻֽנִי׃ </span>|<span class="hb">כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י הָֽעִבְרִ֔ים </span>|<span class="hb">יְהוָה֙ </span>|<span class="hb">כֹּֽה־</span>
8|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=10&verse=3&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 10:3">Exodus 10:3</a> <span class="hb">וַיָּבֹ֨א מֹשֶׁ֣ה וְאַהֲרֹן֮ אֶל־פַּרְעֹה֒ וַיֹּאמְר֣וּ אֵלָ֗יו כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י הָֽעִבְרִ֔ים עַד־מָתַ֣י מֵאַ֔נְתָּ לֵעָנֹ֖ת מִפָּנָ֑י שַׁלַּ֥ח עַמִּ֖י וְיַֽעַבְדֻֽנִי׃ </span>|<span class="hb">כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י הָֽעִבְרִ֔ים </span>|<span class="hb">יְהוָה֙ </span>|<span class="hb">כֹּֽה־</span>
9|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=11&verse=4&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 11:4">Exodus 11:4</a> <span class="hb">וַיֹּ֣אמֶר מֹשֶׁ֔ה כֹּ֖ה אָמַ֣ר יְהוָ֑ה כַּחֲצֹ֣ת הַלַּ֔יְלָה אֲנִ֥י יֹוצֵ֖א בְּתֹ֥וךְ מִצְרָֽיִם׃ </span>|<span class="hb">כֹּ֖ה אָמַ֣ר יְהוָ֑ה </span>|<span class="hb">יְהוָ֑ה </span>|<span class="hb">כֹּ֖ה </span>
10|<a href="https://shebanq.ancient-data.org/hebrew/text?book=Exodus&chapter=32&verse=27&version=2017&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Exodus 32:27">Exodus 32:27</a> <span class="hb">וַיֹּ֣אמֶר לָהֶ֗ם כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י יִשְׂרָאֵ֔ל שִׂ֥ימוּ אִישׁ־חַרְבֹּ֖ו עַל־יְרֵכֹ֑ו עִבְר֨וּ וָשׁ֜וּבוּ מִשַּׁ֤עַר לָשַׁ֨עַר֙ בַּֽמַּחֲנֶ֔ה וְהִרְג֧וּ אִֽישׁ־אֶת־אָחִ֛יו וְאִ֥ישׁ אֶת־רֵעֵ֖הוּ וְאִ֥ישׁ אֶת־קְרֹבֹֽו׃ </span>|<span class="hb">כֹּֽה־אָמַ֤ר יְהוָה֙ אֱלֹהֵ֣י יִשְׂרָאֵ֔ל </span>|<span class="hb">יְהוָה֙ </span>|<span class="hb">כֹּֽה־</span>